In [4]:
''' GettyScraper: Grab the data and images from getty musemun website'''

from bs4 import BeautifulSoup
import requests,sys,urllib.request,os

class GettySpider(object):
    def __init__(self, page):
        self.target = 'http://www.getty.edu/art/collection/search/?pg='+page+'&view=grid&query=YTo1OntzOjE3OiJfZmlsdGVyX2hhc19pbWFnZSI7YjoxO3M6OToiaGFzX2ltYWdlIjtiOjE7czoyMjoiX2ZpbHRlcl9vbl92aWV3X2NlbnRlciI7YjoxO3M6MTM6ImxvY2F0aW9uLnNpdGUiO2E6MTp7aTowO3M6NjoiQ0VOVEVSIjt9czoxMzoiaXNfb25fZGlzcGxheSI7YjoxO30%3D'
        self.names = []
        self.urls = []
        self.nums = 0
        self.items = []
  
    def get_all_url(self):
        req = requests.get(url = self.target)
        html = req.text
        div_bf = BeautifulSoup(html)
        div = div_bf.find_all('ul',class_ = 'small-block-grid-2 medium-block-grid-4 large-block-grid-6')
        fig_bf = BeautifulSoup(str(div))
        fig = fig_bf.find_all('figcaption')
        
        a_bf = BeautifulSoup(str(fig))
        a = a_bf.find_all('a')
        self.nums = len(a)
        for each in a:
            self.names.append(each.string)
            self.urls.append(each.get('href'))
    
    def get_objNum(self, target):
        # get the objNum from url
        start = target.find('objects/')+8
        end = target.find('/',start)
        return target[start:end]
    
    def get_img(self,target):
        
        objNum = self.get_objNum(target)
        
        # get imgs
        req = requests.get(url = target)
        html = req.text
        img_bf = BeautifulSoup(html)
        imgs = img_bf.find_all('img',class_="gallery-image")
        
        if(len(imgs)<1):
            imgs = img_bf.find_all('div',class_="gallery-image")
            if(len(imgs)<1):
                src = None 
            else:
                src = imgs[0].get('data-image-url')
        else:
            src = imgs[0].get('src')
        
        
 
        start = target.find(str(objNum)) + len(str(objNum)) + 1
        end = target.find('/',start)
        filename = str(objNum)+'-'+target[start:end]+'.jpg'
        
        return src,filename
    
    def download_img(self,target):
        src, filename = self.get_img(target)
        if(src != None):
            try:
                urllib.request.urlretrieve(src, 'art_imgs/'+filename)
            except:
                print(src)
                print(filename)
    
    def get_contents(self,target):
        
        objNum = self.get_objNum(target)

        req = requests.get(url = target)
        html = req.text
        bf = BeautifulSoup(html)
        
        obj_detail = bf.find_all('div',class_="medium-9 large-9 columns") 
        
        
        #print(len(obj_detail))
        texts = bf.find_all(class_ = 'justified')

        if len(obj_detail)>0:
            title = obj_detail[0].text.split('(')[0]
            artist = obj_detail[1].text.split('(')[0]
            data_str = obj_detail[3].text+obj_detail[4].text
            date  =''.join([i for i in data_str if i.isdigit() or i == '–'])
        else:
            title = 'No Title'
            artist = 'No Artist'
            date  ='No Date'
        
        if(len(texts)<1):
            content= "No Description"
            sentence = "No Primary Sentence"
        else:
            content = texts[0].text
            sentence = content.split('.')[0]
        src, filename =  self.get_img(target)

        detailDict = {}
        detailDict['object ID'] = objNum 
        detailDict['Title'] = title
        detailDict['Artist/Maker'] = artist
        detailDict['Date'] = date

        detailDict['Description'] = content
        detailDict['Primary Sentence'] = sentence
        detailDict['# of People'] = ''
        detailDict['Gender'] = ''
        detailDict['Age'] = ''
        detailDict['Emotion'] = ''
        detailDict['Image'] = filename

        self.items.append(detailDict)
        return self.items


data = []
gs_list = []


'''=========================Step one========================='''
# Change the page numbers you are going to grab, and run this block
startPage=30
endPage = 36

for p in range(startPage,endPage+1):
  #  p = 16
    gs = GettySpider(str(p))
    gs.get_all_url()
    gs_list.append(gs)
    
    temp = []
    print("Start Scraping Data from Page: %d"%p)
    for i in range(gs.nums):
        temp = list(gs.get_contents(gs.urls[i]))
        sys.stdout.write("Scraping:%.3f%%" %  float((i/gs.nums)*100) + '\r')
    data.append(temp)
    print("Finish Scraping from Page %d!!!!!!!!!!!! "%p)
    
    
    
    

Start Scraping Data from Page: 30
Finish Scraping from Page 30!!!!!!!!!!!! 
Start Scraping Data from Page: 31
Finish Scraping from Page 31!!!!!!!!!!!! 
Start Scraping Data from Page: 32
Finish Scraping from Page 32!!!!!!!!!!!! 
Start Scraping Data from Page: 33
Finish Scraping from Page 33!!!!!!!!!!!! 
Start Scraping Data from Page: 34
Finish Scraping from Page 34!!!!!!!!!!!! 
Start Scraping Data from Page: 35
Finish Scraping from Page 35!!!!!!!!!!!! 
Start Scraping Data from Page: 36
Finish Scraping from Page 36!!!!!!!!!!!! 


In [5]:
'''=========================Step Two========================='''
# 1. After finishing scraping, Creating a new folder called 'art_imgs' in the current directory
# 2. run this block will download the images of your pages and rename it into the right format

print("Start Downloading Image")
for i in range(len(data)):
    for j in range(len(data[i])):
        gs_list[i].download_img(gs_list[i].urls[j])
        sys.stdout.write(" Downloaded:%.3f%%" %  float(i/len(data)) + '\r')
        
print("Finish Downloading Image!!!!!!!!!!!!")       

Start Downloading Image
http://www.getty.edu/museum/media/images/web/enlarge/00616601.jpg
5450-after-painting-by-francois-boucher-and-after-designs-by-maurice-jacques-and-after-designs-by-louis-tessier-et-al-tapestry-jupiter-transforme-en-diane-pour-surprendre-callisto-avec-vertumne-et-pomone-from-les-tentures-de-francois-boucher-series-french-1776-1778.jpg
Finish Downloading Image!!!!!!!!!!!!


In [6]:
'''=========================Step Three========================='''
# 1. This block of code is to filter the images that are useless
# 2. So what you should do is to manual delete the imgs that don't have human in it from the directory 'art_imgs'
# 3. Run this block of code to append useful images into a newData list

newData=[]
for i in range(len(data)):
    for j in range(len(data[i])):
        src,filename = gs_list[i].get_img(gs_list[i].urls[j])
        path ='art_imgs/'+filename
        if os.path.exists(path):  
            newData.append(data[i][j])
            print("Found the File,appended to newData list")    
        else:  
            print("Not Found the File" )    
# print(newData)



Not Found the File
Found the File,appended to newData list
Found the File,appended to newData list
Not Found the File
Not Found the File
Not Found the File
Not Found the File
Not Found the File
Not Found the File
Not Found the File
Found the File,appended to newData list
Not Found the File
Found the File,appended to newData list
Found the File,appended to newData list
Not Found the File
Found the File,appended to newData list
Not Found the File
Not Found the File
Found the File,appended to newData list
Found the File,appended to newData list
Not Found the File
Not Found the File
Found the File,appended to newData list
Found the File,appended to newData list
Found the File,appended to newData list
Found the File,appended to newData list
Found the File,appended to newData list
Not Found the File
Found the File,appended to newData list
Found the File,appended to newData list
Found the File,appended to newData list
Not Found the File
Found the File,appended to newData list
Found the File,a

In [8]:
'''=========================Final Step========================='''
# 1. Run the following code to format it into a table and Export to excel

# 2. few of data might be missing because it doesn't have data 
#    or can't acess the data from the website, need to fix it mannully
 
# 3. For '# of People' 'Gender' 'Age' 'Emotion' program is not smarter enough to scrap, 
#    need to fill into table mannully


import pandas as pd

df = pd.DataFrame(newData)
df = df[['object ID', 'Title','Artist/Maker','Date','Description','Primary Sentence','# of People','Gender','Age','Emotion','Image']]
df.to_excel('art_data.xls')

display(df)



,object ID,Title,Artist/Maker,Date,Description,Primary Sentence,# of People,Gender,Age,Emotion,Image
0,568,Arrival of Saint Ursula at Cologne,Bernardo Daddi,1333,Three boats approach the German city of Cologn...,Three boats approach the German city of Cologn...,,,,,568-bernardo-daddi-arrival-of-saint-ursula-at-...
1,813,The Annunciation,Paolo Veneziano,1348–1350,These panels once formed the pinnacles of the ...,These panels once formed the pinnacles of the ...,,,,,813-paolo-veneziano-the-annunciation-italian-a...
2,601,Saint John Baptizing in the River Jordan,Nicolas Poussin,1630,"""My temperament compels me to look for and tak...","""My temperament compels me to look for and tak...",,,,,601-nicolas-poussin-saint-john-baptizing-in-th...
3,223220,Lucretia,Workshop of Carl von Egeri,1561,The suicide of Lucretia is represented next to...,The suicide of Lucretia is represented next to...,,,,,223220-workshop-of-carl-von-egeri-lucretia-swi...
4,615,The Supper at Emmaus,Bartolomeo Cavarozzi,1615–1625,"As the innkeeper looks on, the resurrected Chr...","As the innkeeper looks on, the resurrected Chr...",,,,,615-bartolomeo-cavarozzi-the-supper-at-emmaus-...
5,5925,Pair of Busts: Louis XV and Marie Leczinska,Rue de Charenton Manufactory,1745–1755,A short-lived Parisian earthenware manufactory...,A short-lived Parisian earthenware manufactory...,,,,,5925-rue-de-charenton-manufactory-paris-or-cha...
6,596,Venus and Mars,Palma il Giovane,1605–1609,Venus twists on the bed in a curved yet inviti...,Venus twists on the bed in a curved yet inviti...,,,,,596-palma-il-giovane-venus-and-mars-italian-ab...
7,129847,Christ on the Cross,El Greco,1600–1610,"In this painting, El Greco combines the iconic...","In this painting, El Greco combines the iconic...",,,,,129847-el-greco-domenico-theotokopoulos-christ...
8,763,Shepherd with a Flute,Giovanni Girolamo Savoldo,1525,Little is known about the life of this Italian...,Little is known about the life of this Italian...,,,,,763-giovanni-girolamo-savoldo-shepherd-with-a-...
9,800,The Doctor's Visit,Frans van Mieris the Elder,1667,In the foreground of this minutely detailed pa...,In the foreground of this minutely detailed pa...,,,,,800-frans-van-mieris-the-elder-the-doctor's-vi...
